In [1]:
import pandas as pd
import talib
from enum import Enum
import QuantLib as ql
from market import MarketData
from datetime import date 
from pricing import *

old

In [2]:
class Indicator:
    def __init__(self):
        self.methods = {}

    def register(self, name=None):
        """
        A decorator to register an indicator method with a custom or derived name.
        """
        def decorator(func):
            indicator_name = name or func.__name__
            self.methods[indicator_name] = func
            return func
        return decorator

    def compute(self, name, *args, **kwargs):
        """
        Compute the registered indicator.
        """
        if name not in self.methods:
            raise ValueError(f"Indicator '{name}' is not registered.")
        return self.methods[name](*args, **kwargs)


class Backtester:
    def __init__(self, history: pd.DataFrame, starting_balance: float,):
        self.starting_balance = starting_balance
        self.position_open = False
        self.equity_history = []
        self.signals = []
        self.indicators = Indicator()  # Indicator object for registration
        self._add_data(history)
        self.indicator_values = pd.DataFrame(index=history.index)

    def _add_data(self, history):
        self._history = history

    def backtest(self):
        """ chiama in automatico on_data a ogni iterazione e aggiorna history con i valori noti fino alla data"""
        for i in range(0, len(self._history)):
            self.history = self._history.iloc[:i + 1]
            self.on_data()

    def add_indicator(self, name, value):
        """
        Automatically updates the indicator DataFrame.
        """
        self.indicator_values.loc[self.history.index[-1], name] = value

    def get_data_at_index(self, idx):
        """
        Returns the historical data and indicators up to the specified index.
        :param idx: The index yyyy-mm-dd up to which to return data.
        :return: A combined DataFrame with historical data and indicators.
        """
        if idx not in self._history.index:
            raise ValueError(f"Index {idx} is not in the historical data.")
        
        # Slice the historical data and indicators up to the specified index
        data = self._history.loc[:idx]
        indicators = self.indicator_values.loc[:idx]
        
        # Combine historical data and indicators into one DataFrame
        return pd.concat([data, indicators], axis=1)
    
    def execute_trade(self, trade_type, quantity, price):
        """Esegue operazioni di acquisto o vendita."""
        cost = quantity * price

        if trade_type == OrderType.BUY:
            if self.starting_balance >= cost:
                self.starting_balance -= cost
                self.position_open = True
                self.signals.append({"type": "BUY", "price": price})
            else:
                print("Fondi insufficienti per comprare.")
        elif trade_type == OrderType.SELL:
            if self.position_open:
                self.starting_balance += cost
                self.position_open = False
                self.signals.append({"type": "SELL", "price": price})
            else:
                print("Nessuna posizione aperta da vendere.")

        # Aggiorna l'equity history
        self.equity_history.append(self.starting_balance)

    def on_data(self):
        """
        Implement trading logic.
        """
        pass


class BacktesterConcrete(Backtester):
    def __init__(self, history, starting_balance, ema_window=20):
        super().__init__(history, starting_balance)
        self.ema_window = ema_window

        # Register EMA indicator
        @self.indicators.register(name="ema")
        def ema(series, window):
            return talib.EMA(series, timeperiod=window)

    def on_data(self):
        """Calcola l'EMA e genera segnali di trading."""
        latest_price = self.history["price"].iloc[-1]

        # Calcola EMA
        ema_series = self.indicators.compute("ema", self.history["price"], self.ema_window)
        latest_ema = ema_series.iloc[-1] if not ema_series.isna().iloc[-1] else None

        # Salva EMA nel DataFrame indicator_values
        self.add_indicator("ema", latest_ema)

        # Genera segnali di trading
        if latest_ema is not None:
            if latest_price > latest_ema and not self.position_open:
                self.execute_trade(OrderType.BUY, quantity=1, price=latest_price)
            elif latest_price < latest_ema and self.position_open:
                self.execute_trade(OrderType.SELL, quantity=1, price=latest_price)


# Pricing

In [3]:
symbol = 'ctp1'
start_date = date(2022, 10, 14)
end_date = date(2023, 10, 12)
market = MarketData(start_date, end_date)

market.build(symbol)

## Equity

In [4]:
entry_mkt = market.market_data[date(2022, 10, 14)]
exit_mkt = market.market_data[date(2022, 11, 14)]

equity_position = EquityPosition(symbol, quantity=10, market_data=entry_mkt, position_type=PositionType.LONG)
print("entry ref: ", entry_mkt['equity'][symbol]['price'])

entry ref:  1.7124


In [5]:
print("exit ref: ", exit_mkt['equity'][symbol]['price'])
print("PNL :", equity_position.calculate_pnl(exit_mkt))


exit ref:  2.1865
PNL : 4.741000000000001


In [23]:
equity_position.trade_id

32137

## Option

In [6]:
quantity= 1
strike_price= 1.7
expiry_date= date(2023, 9, 18)
option_type= OptionType.CALL
position_type= PositionType.LONG

In [7]:
opt_position = OptionPosition(symbol,
               5,
               entry_mkt,
               strike_price,
               expiry_date,
               option_type,
               position_type)

In [8]:
opt_position.calculate_value(entry_mkt)

0.1479706636063528

In [9]:
opt_position.calculate_value(exit_mkt)

0.5400440794388559

In [10]:
opt_position.calculate_pnl(exit_mkt)

1.9603670791625154

In [11]:
(opt_position.calculate_value(exit_mkt) - opt_position.calculate_value(entry_mkt) )*5

1.9603670791625154

pnl at expiry

In [12]:
from datetime import timedelta

In [13]:
at_expiry_mkt = market.market_data[expiry_date]
opt_position.calculate_value(at_expiry_mkt)

0.7300000000000002

In [14]:
at_expiry_mkt = market.market_data[expiry_date + timedelta(1)]
opt_position.calculate_value(at_expiry_mkt)

0

# Portfolio

In [15]:
ptf = Portfolio()
ptf.add_position(equity_position)
ptf.add_position(opt_position)

In [16]:
position_df = pd.DataFrame()

In [17]:
a = pd.DataFrame(ptf.get_positions_summary(exit_mkt))
a1 = pd.DataFrame(ptf.get_positions_summary(exit_mkt))

In [18]:
position_df = pd.concat([position_df, a], axis = 0)

In [19]:
position_df

,ref_date,trade_date,symbol,type,side,quantity,entry_price,current_value,open_pnl,closed_pnl,global_pnl
0,2022-11-14,2022-10-14,ctp1,equity,1,10,1.712400,2.186500,4.741,0,4.741
1,2022-11-14,2022-10-14,ctp1,option,1,0,0.147971,0.540044,0.000,0,0.000


In [20]:
a1

,ref_date,trade_date,symbol,type,side,quantity,entry_price,current_value,open_pnl,closed_pnl,global_pnl
0,2022-11-14,2022-10-14,ctp1,equity,1,10,1.712400,2.186500,4.741,0,4.741
1,2022-11-14,2022-10-14,ctp1,option,1,0,0.147971,0.540044,0.000,0,0.000


In [21]:
ptf.calculate_total_pnl(exit_mkt)

AttributeError: 'Portfolio' object has no attribute 'calculate_total_pnl'

# Backtest

In [ ]:
# params 
df_prices = pd.read_csv('data/prices/isp_prices.csv')
params = {
    "starting_balance": 10000,
    "ema_window": 20
}

FileNotFoundError: [Errno 2] No such file or directory: 'data/prices/isp_prices.csv'

In [ ]:
backtester  = BacktesterConcrete(df_prices, **params)
backtester.backtest()